In [1]:
from matplotlib import colors
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import tensorflow as tf
from scipy.misc import logsumexp
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
# The digits dataset


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
class TFNaiveBayesClassifier:
    dist = None

    def fit(self, X, y):
        # Separate training points by class (nb_classes * nb_samples * nb_features)
        unique_y = np.unique(y)
        points_by_class = [
            [x for x, t in zip(X, y) if t == c]
            for c in unique_y]
        minsize = 100000
        for p in points_by_class:
            minsize = min(minsize, len(p))
        for i, p in enumerate(points_by_class):
            points_by_class[i] = points_by_class[i][:minsize]
        points_by_class = np.array(points_by_class)
        print(points_by_class.shape)
        # Estimate mean and variance for each class / feature
        # shape: nb_classes * nb_features
        mean, var = tf.nn.moments(tf.constant(points_by_class), axes=[1])

        # Create a 3x2 univariate normal distribution with the 
        # known mean and variance
        self.dist = tf.distributions.Normal(loc=mean, scale=tf.sqrt(var))

    def predict(self, X):
        assert self.dist is not None

        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # Conditional probabilities log P(x|c) with shape
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(
            self.dist.log_prob(
                tf.reshape(
                    tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features])),
            axis=2)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes))

        # posterior log probability, log P(c) + log P(x|c)
        cond_probs = tf.cast(cond_probs, tf.float64)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keep_dims=True)
        log_prob = joint_likelihood - norm_factor
        # exp to get the actual probabilities
        return tf.exp(log_prob)

In [7]:
X = mnist.train.images[:3000,200:300]+1e-8
y = mnist.train.labels[:3000]

tf_nb = TFNaiveBayesClassifier()
tf_nb.fit(X, y)


s = tf.Session()
Z = s.run(tf_nb.predict(X))
predicted_label = np.argmax(Z, axis=1)

(10, 264, 100)


In [8]:
predicted_label

array([7, 1, 2, ..., 2, 2, 8])

In [9]:
np.mean(y==predicted_label)

0.21533333333333332